In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
train['lgb_pred'] = np.load("../input/30d-ml-2021-lgb-no-pseudo/lgb_oof.npy")
test['lgb_pred'] = np.load("../input/30d-ml-2021-lgb-no-pseudo/lgb_pred.npy")

train['xgb_pred'] = np.load("../input/30d-ml-2021-xgb-no-pseudo/xgb_oof.npy")
test['xgb_pred'] = np.load("../input/30d-ml-2021-xgb-no-pseudo/xgb_pred.npy")

train['xgb2_pred'] = np.load("../input/30d-ml-2021-xgb-model-2-no-pseudo/xgb_oof.npy")
test['xgb2_pred'] = np.load("../input/30d-ml-2021-xgb-model-2-no-pseudo/xgb_pred.npy")

train['ctb_pred'] = np.load("../input/30d-ml-2021-catb-no-pseudo/ctb_oof.npy")
test['ctb_pred'] = np.load("../input/30d-ml-2021-catb-no-pseudo/ctb_pred.npy")

In [6]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont8,cont9,cont10,cont11,cont12,cont13,lgb_pred,xgb_pred,xgb2_pred,ctb_pred
0,B,B,B,C,B,B,A,E,C,N,...,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.438170,8.425334,8.430861,8.420048
1,B,B,A,A,B,D,A,F,A,O,...,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.361583,8.361334,8.328455,8.375351
2,A,A,A,C,B,D,A,D,A,F,...,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.174809,8.177517,8.190731,8.143454
3,B,B,A,C,B,D,A,E,C,K,...,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.376340,8.398763,8.401320,8.357306
4,A,A,A,C,B,D,A,E,A,N,...,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,8.140109,8.256151,8.255340,8.248562


# Preprocessing

In [7]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()
'''

"\n# List of categorical columns\nobject_cols = [col for col in features.columns if 'cat' in col]\n\n# ordinal-encode categorical columns\nX = features.copy()\nX_test = test.drop(['id'], axis=1).copy()\nordinal_encoder = OrdinalEncoder()\nX[object_cols] = ordinal_encoder.fit_transform(features[object_cols])\nX_test[object_cols] = ordinal_encoder.transform(test[object_cols])\n\n# Preview the ordinal-encoded features\nX.head()\n"

In [8]:

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()


,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,ohe_46,ohe_47,ohe_48,ohe_49,ohe_50,ohe_51,ohe_52,ohe_53,ohe_54,ohe_55
0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [10]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Optuna

In [11]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [12]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=X, y=y):
  """
  """
  param_space = {
               #'device':'gpu',  # Use GPU acceleration
               #'boosting_type': 'gbdt',
               'reg_lambda':trial.suggest_uniform('reg_lambda', 0.01, 25),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 7, 31),
                'subsample': trial.suggest_uniform('subsample', 0.513, 0.913),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.601, 1.0),
                #'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
               'learning_rate':trial.suggest_uniform('learning_rate', 1e-2, 1e-1),
                 'min_child_samples':trial.suggest_int('min_child_samples', 2, 45),
              'num_leaves':trial.suggest_int('num_leaves', 5, 60),
              'max_depth':trial.suggest_int('max_depth', 20, 34),
              #'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.005),
              #'class_weight':trial.suggest_categorical('class_weight',['balanced',None]),
               'n_estimators':N_ESTIMATORS,
                'n_jobs' : -1,
              'metric':'rmse',
              'max_bin':trial.suggest_int('max_bin', 399, 799),
              'cat_smooth':trial.suggest_int('cat_smooth', 32, 82),
              'cat_l2':trial.suggest_loguniform('cat_l2', 1e-3, 50)
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  #lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  pred=np.zeros(y_va.shape[0])
    
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
    
  #X_tr = apply_noise_row(X_tr)

  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = lgb.LGBMRegressor(**param_space)
    model.fit(
        X_tr, 
        y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
        categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
        #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
    )


    pred += model.predict(X_va) / len(seed_list)
    
  #elapsed = time.time() - start
  rmse = mean_squared_error(y_va, pred, squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [13]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-28 10:26:18,458] A new study created in memory with name: no-name-1661ef11-0a84-4957-bb07-360f0d453377


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[163]	valid_0's rmse: 0.713839
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[157]	valid_0's rmse: 0.713815


[I 2021-08-28 10:26:31,731] Trial 0 finished with value: 0.713817130596516 and parameters: {'reg_lambda': 21.32265451668209, 'reg_alpha': 9.490127657609712, 'subsample': 0.6183308013887656, 'colsample_bytree': 0.6508665905956785, 'learning_rate': 0.02830386627318216, 'min_child_samples': 28, 'num_leaves': 22, 'max_depth': 31, 'max_bin': 526, 'cat_smooth': 65, 'cat_l2': 21.69646543711112}. Best is trial 0 with value: 0.713817130596516.


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[69]	valid_0's rmse: 0.713772
Training until validation scores don't improve for 200 rounds


[I 2021-08-28 10:26:41,886] Trial 1 finished with value: 0.713762641025865 and parameters: {'reg_lambda': 19.094788397260594, 'reg_alpha': 29.164994475889145, 'subsample': 0.6874427872007026, 'colsample_bytree': 0.8082250080284827, 'learning_rate': 0.07500664489612797, 'min_child_samples': 34, 'num_leaves': 15, 'max_depth': 20, 'max_bin': 438, 'cat_smooth': 80, 'cat_l2': 2.0370622967946463}. Best is trial 1 with value: 0.713762641025865.


...

Early stopping, best iteration is:
[46]	valid_0's rmse: 0.71385
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[150]	valid_0's rmse: 0.713731
Training until validation scores don't improve for 200 rounds


[I 2021-08-28 10:31:45,273] Trial 28 finished with value: 0.7137288841521282 and parameters: {'reg_lambda': 15.75314850268768, 'reg_alpha': 26.72119952819424, 'subsample': 0.769157283091034, 'colsample_bytree': 0.8136158991561, 'learning_rate': 0.03782786095556265, 'min_child_samples': 23, 'num_leaves': 6, 'max_depth': 24, 'max_bin': 471, 'cat_smooth': 74, 'cat_l2': 0.002066976922512104}. Best is trial 14 with value: 0.7137002996077744.


Early stopping, best iteration is:
[152]	valid_0's rmse: 0.713733
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[349]	valid_0's rmse: 0.713732
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[359]	valid_0's rmse: 0.713721


[I 2021-08-28 10:31:57,741] Trial 29 finished with value: 0.7137236761140465 and parameters: {'reg_lambda': 21.721981521240462, 'reg_alpha': 28.658854898375782, 'subsample': 0.6272334216791822, 'colsample_bytree': 0.8477791398145609, 'learning_rate': 0.017636098152189764, 'min_child_samples': 30, 'num_leaves': 5, 'max_depth': 30, 'max_bin': 549, 'cat_smooth': 66, 'cat_l2': 0.0050173896798538275}. Best is trial 14 with value: 0.7137002996077744.


Number of finished trials: 30
Best trial: {'reg_lambda': 17.15595263626255, 'reg_alpha': 28.41175647691953, 'subsample': 0.8872494576691315, 'colsample_bytree': 0.8822495475460936, 'learning_rate': 0.0840589671215061, 'min_child_samples': 26, 'num_leaves': 5, 'max_depth': 28, 'max_bin': 598, 'cat_smooth': 71, 'cat_l2': 0.004758350233688687}


In [14]:
study.best_params

{'reg_lambda': 17.15595263626255,
 'reg_alpha': 28.41175647691953,
 'subsample': 0.8872494576691315,
 'colsample_bytree': 0.8822495475460936,
 'learning_rate': 0.0840589671215061,
 'min_child_samples': 26,
 'num_leaves': 5,
 'max_depth': 28,
 'max_bin': 598,
 'cat_smooth': 71,
 'cat_l2': 0.004758350233688687}

# Log

====== Ordinal encoding =========

0.7137357370735623 no noise ver1

====== One-hot encoding =========

0.7137291748487112 no noise ver2

narrow